In [19]:
# Workaway.py
from pickle import FALSE
import time
from operator import contains
from select import select
from tomlkit import key_value
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import numpy as np

#DIFFERENT METHODS OF ACCESSING ELEMENTS
#print(match_internet_access.get_attribute('innerHTML'))
#print(match_internet_access.get_attribute('innerText'))
# print(match_internet_access.get_property)
# print(match_internet_access.text)
# print(match_internet_access.accessible_name)
# print(match_internet_access.tag_name)
### Continue with first tab
#driver.switch_to.window(driver.window_handles[0])
### Scroll the page
#driver.execute_script("window.scrollTo(0,1800)")
### Handling objects in frames (iframe or frame doesn't matter)
#iframe = driver.find_element(By.XPATH, "//iframe[contains(@name, 'uspapi')]")
#driver.switch_to.frame(iframe)
### Find elements by Class name
#match_host_rating = driver.find_element(By.CLASS_NAME, 'hr_100 inline-block')


##### INPUT AREA #####
website = 'https://www.workaway.info/en/hostlist'
countries = pd.read_csv("Countries.csv")
#countries = countries[107:] #195,196=United States
#countries.loc[0].at["country"] = "Angola" #delete
print(countries) #delete
feedback = 0
lastminute = 0
##### END OF INPUT AREA #####

def configure_driver():
    chrome_options = webdriver.ChromeOptions()
    ### Disable displaying images
    #prefs = {"profile.managed_default_content_settings.images": 2}
    #chrome_options.add_experimental_option("prefs", prefs)
    ### Maximize window
    chrome_options.add_argument("--start-maximized")
    ### Add the argument and make the browser Headless.
    chrome_options.add_argument("--headless")
    ### Instantiate the Webdriver
    driver = webdriver.Chrome(options = chrome_options)
    return driver

### Extracting function on hostlist page
def extract_element(driver, 
                    my_timer, 
                    element, 
                    path, 
                    extract_attribute = 'text', 
                    attribute_version = 1, 
                    attribute_value = "src"):
    if attribute_version == 0:
        try:
            match_temp = WebDriverWait(driver, my_timer).until(EC.presence_of_all_elements_located((By.XPATH, path)))
            match = len(match_temp)
        except TimeoutException:
            print("TimeoutException: photos not found")
            match = np.nan
    elif attribute_version == 3:
        #h = []
        match = []
        try:
            match_temp = WebDriverWait(driver, my_timer).until(EC.presence_of_all_elements_located((By.XPATH, path)))
            #all_spans = driver.find_elements_by_xpath("//span[@class='class']")
            #for span in all_spans:
            #print all.text
            for match_i in match_temp:
                match.append(getattr(match_i, extract_attribute))
                #match_h = getattr(match_i, extract_attribute)
                #h.append(match_h)
            #match = h
        except TimeoutException:
            print("TimeoutException: comments not found")
            match = np.nan
    else:
        try:
            match_temp = WebDriverWait(driver, my_timer).until(EC.presence_of_element_located((By.XPATH, path)))
            if attribute_version == 2:
                try:
                    match = getattr(match_temp, extract_attribute)(attribute_value)
                except:
                    print("New error '2' to handle")
            elif attribute_version == 1:
                try:
                    match = getattr(match_temp, extract_attribute)
                except:
                    print("New error '1' to handle")
        except TimeoutException:
            #print("TimeoutException: paragraph not found")
            #print(f"{element=}")
            match = np.nan
    element.append(match)
    #print(element)


### Open the Workaway finding host website
driver = configure_driver()
driver.get(website)

### COOKIES ACCEPT
cookie_accept = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'CybotCookiebotDialogBodyButtonAccept')))
#cookie_accept = driver.find_element(By.ID, 'CybotCookiebotDialogBodyButtonAccept') #old version
driver.execute_script("arguments[0].click();", cookie_accept)

### Check "only with feedback" checkbox
if feedback == 1 and (driver.find_element(By.ID, 'with_feedback').is_selected()) != True:
    driver.find_element(By.ID, 'with_feedback').is_selected()
    feedback_checkbox = driver.find_element('xpath', '//label[@for="with_feedback"]/descendant::div')
    feedback_checkbox.click()


### LOOP THROUUGH COUNTRIES IN THE CSV LIST
for country in countries['country']:
    ### Input the name of country/region/continent + activate
    try:
        searchbar = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//input[@placeholder="Anywhere"]')))
    except TimeoutException:
        print(f'INFO: Looks like you broken the site during {country} search')
        driver.quit() #this was added to close the previous driver - TO BE TESTED
        driver.get(website)
        continue
    searchbar.click()
    time.sleep(2)
    ### Alternative clicking type on searchbar
    #searchbar = driver.find_element('xpath', '//input[@placeholder="Anywhere"]')
    #driver.execute_script("arguments[0].click();", searchbar)
    print(f'printing just before sending the command to searchbox {country}')
    searchbar.clear()
    searchbar.send_keys(country) 
    time.sleep(2)
    searchbar.send_keys(Keys.ARROW_DOWN)
    searchbar.send_keys(Keys.ENTER)
    ### Handling of optional visa popup - United Kingdom and United States //newly added try/except block
    try:
        visabutton = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '//button[@data-action="visaagree"]')))
        visabutton.click()
    except TimeoutException:
        print("No Visa popup to handle")    ### Find first host on the Listentry Title object and open it
    try:
        hostlist = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '//div[@class="listentry-content"]/child::a')))
    except TimeoutException:
        continue
    driver.execute_script("arguments[0].click();", hostlist)
    #hostlist.click()
    
    ### Handling opened tabs
    p = driver.window_handles[0]
    #obtain browser tab window
    c = driver.window_handles[1]
    driver.switch_to.window(c)
    try:
        visabutton = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '//button[@data-action="visaagree"]')))
        visabutton.click()
    except TimeoutException:
        print("No Visa popup to handle")
    ### WEBSCRAPING THE HOST DATA ###
    ### For LOOP to iterate over number of found hosts
    ### Lists initialization
    title = [] #
    country_host = [] #
    host_rating = [] #
    feedback_no = [] #
    h_comm = [] #
    ww_comm = [] #
    last_activity = [] #
    reply_rate = [] #
    average_reply_time = [] #
    favourited = [] #
    min_stay = [] #
    description = [] #
    types_of_help = [] #
    cultural_exchange = [] #
    help = [] #
    languages_spoken = [] #
    accommodation = [] #
    what_else = [] #
    internet_access = [] #
    limited_internet = [] #
    pets = [] #
    smokers = [] #
    hosting_families = [] #
    digital_nomad = [] #
    campervans = [] #
    possibly_pets = [] #
    workawayer_total = [] #
    hours_expected = [] #
    number_of_photos = [] #
    feedback_accuracy = [] #
    feedback_cultural_exchange = [] #
    feedback_communication = [] #

    i = 1
    while i < 20000:
        extract_element(driver, 10, title, '//div[@class="details"]//child::h1', 'text')
        extract_element(driver, 1, country_host, '//div[@class="country-row"]//child::p', 'text')
        extract_element(driver, 1, host_rating, '//div[contains(@class, "hr")]', 'get_attribute', attribute_version = 2, attribute_value = "class")
        extract_element(driver, 1, feedback_no, '//h2[text()="Feedback"]//parent::div//following-sibling::div[1]//p', 'text')
        extract_element(driver, 1, last_activity, '//*[text()="Last activity"]//following::div[1]//child::p', 'text')
        extract_element(driver, 1, reply_rate, '//*[text()="Reply rate"]//following::div[1]//child::p', 'text')
        extract_element(driver, 1, average_reply_time, '//*[text()="Average reply time:"]//following::div[1]//child::p', 'text')
        extract_element(driver, 1, favourited, '//p[@class="blue-text"]', 'text')
        extract_element(driver, 1, min_stay, '//p[@class="text-muted"]', 'text')
        extract_element(driver, 1, description, '(//div[@id="section-information"]//descendant::p)[1]', 'text')
        extract_element(driver, 1, types_of_help, '(//div[@id="section-information"]//descendant::p)[2]', 'text')
        extract_element(driver, 1, internet_access, '//*[text()="Internet access"]//parent::div//parent::li//img[contains(@src, "")]', 'get_attribute', attribute_version = 2, attribute_value = "src")
        extract_element(driver, 1, cultural_exchange, '//img[@alt="Cultural exchange and learning opportunities"]//parent::li//child::p', 'text')
        extract_element(driver, 1, help, '//img[@alt="Help"]//parent::li//child::p', 'text')
        extract_element(driver, 1, languages_spoken, '//h2[contains(text(), "Languages")]//parent::div//child::p', 'text')
        extract_element(driver, 1, accommodation, '//img[@alt="Accommodation"]//parent::li//child::p', 'text')
        extract_element(driver, 1, what_else, '//img[@alt="What else ..."]//parent::li//child::p', 'text')
        extract_element(driver, 1, limited_internet, '//*[text()="Limited internet access"]//parent::div//parent::li//child::img', 'get_attribute', attribute_version = 2, attribute_value = "src")
        extract_element(driver, 1, pets, '//*[text()="We have pets"]//parent::div//parent::li//child::img', 'get_attribute', attribute_version = 2, attribute_value = "src")
        extract_element(driver, 1, smokers, '//*[text()="We are smokers"]//parent::div//parent::li//child::img', 'get_attribute', attribute_version = 2, attribute_value = "src")
        extract_element(driver, 1, hosting_families, '//*[text()="Can host families"]//parent::div//parent::li//child::img', 'get_attribute', attribute_version = 2, attribute_value = "src")
        extract_element(driver, 1, digital_nomad, '//*[text()="Can host digital nomads"]//parent::div//child::p', 'text')
        extract_element(driver, 1, campervans, '//*[text()="Space for parking camper vans"]//parent::div//child::p', 'text')
        extract_element(driver, 1, possibly_pets, '//*[text()="Can possibly accept pets"]//parent::div//child::p', 'text')
        extract_element(driver, 1, workawayer_total, '//*[text()="How many Workawayers can stay?"]//parent::div//child::p', 'text')
        extract_element(driver, 1, hours_expected, '//*[text()="Hours expected"]//parent::div//child::p', 'text')
        extract_element(driver, 1, number_of_photos, '//div[contains(@id, "pic")]', attribute_version = 0)
        extract_element(driver, 1, feedback_accuracy, '(//*[text()="These are extra optional ratings when members leave feedback. The average rating left for each option is displayed."]//following::p)[1]', 'text')
        extract_element(driver, 1, feedback_cultural_exchange, '(//*[text()="These are extra optional ratings when members leave feedback. The average rating left for each option is displayed."]//following::p)[2]', 'text')
        extract_element(driver, 1, feedback_communication, '(//*[text()="These are extra optional ratings when members leave feedback. The average rating left for each option is displayed."]//following::p)[3]', 'text')
        extract_element(driver, 1, ww_comm, '//div[@class="feedback_content feedback_content_ww"]/parent::div/following-sibling::div[@class="feedback-thumb-wrapper"]/span', 'text', attribute_version = 3)
        extract_element(driver, 1, h_comm, '//div[@class="feedback_content feedback_content_h"]/parent::div/following-sibling::div[@class="feedback-thumb-wrapper"]/span', 'text', attribute_version = 3)


        ### Go to NEXT host
        try:
            next_host = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//a[@title="Next in search result" or contains(@title, "Next listing")]')))
        except TimeoutException:
            print("TimeoutException: END OF HOST LIST")
            break
        next_host.click()
        i += 1

    print(f"Total number of hosts webscraped in {country} = {i-1}")
    ### Creating DF and exporting to ';' delimited CSV file
    df = pd.DataFrame({'title': title,
        'country_host': country_host,
        'host_rating': host_rating,
        'ww_comm': ww_comm,
        'h_comm': h_comm,
        'feedback_no': feedback_no,
        'last_activity': last_activity,
        'reply_rate': reply_rate,
        'average_reply_time': average_reply_time,
        'favourited': favourited,
        'min_stay': min_stay,
        'description': description,
        'types_of_help': types_of_help,
        'cultural_exchange': cultural_exchange,
        'help': help,
        'languages_spoken': languages_spoken,
        'accomodation': accommodation,
        'what_else': what_else,
        'internet_access': internet_access,
        'limited_internet': limited_internet,
        'pets': pets, 
        'smokers': smokers,
        'hosting_families': hosting_families,
        'digital_nomad': digital_nomad,  
        'campervans':  campervans,
        'possibly_pets': possibly_pets,
        'workawayer_total': workawayer_total,
        'hours_expected': hours_expected,
        'number_of_photos': number_of_photos,
        'feedback_accuracy': feedback_accuracy,
        'feedback_cultural_exchange': feedback_cultural_exchange,
        'feedback_communication': feedback_communication})
    #print(df)
    saving_path = f'workaway_scraping_{country}.csv'
    df.to_csv(saving_path, sep =';', index = False)
    driver.close()
    driver.switch_to.window(p)

### Time: 70m - Egipt:Senegal - 17.08.2022
### Time: 1545m - Tanzania:Lichtenstein - 17.08.2022]
driver.quit()

### Ordinary SELENIUM EXPRESSIONS TO OBTAIN THE DATA
#match_title = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="details"]//child::h1')))
# #print(match_title.text)
# match_country_host = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="country-row"]//child::p')))
# print(match_country_host.text)
# match_host_rating = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "hr")]')))
# print(match_host_rating.get_attribute('class'))
# match_last_activity = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[text()="Last activity"]//following::div[1]//child::p')))
# print(match_last_activity.text)
# #matched_reply_rate = driver.find_element(By.XPATH, '//*[text()="Reply rate"]//following::div[1]//child::p')
# match_reply_rate = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[text()="Reply rate"]//following::div[1]//child::p')))
# print(match_reply_rate.text)
# match_reply_time = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[text()="Average reply time:"]//following::div[1]//child::p')))
# print(match_reply_time.text)
# match_favourited = driver.find_element(By.XPATH, '//p[@class="blue-text"]')
# print(match_favourited.text)
# match_min_stay = driver.find_element(By.XPATH, '//p[@class="text-muted"]')
# print(match_min_stay.text)
# match_description = driver.find_element(By.XPATH, '(//div[@id="section-information"]//descendant::p)[1]')
# print(match_description.text)
# match_types_of_help = driver.find_element(By.XPATH, '(//div[@id="section-information"]//descendant::p)[2]')
# print(match_types_of_help.text)
# match_cultural_exchange = driver.find_element(By.XPATH, '//img[@alt="Cultural exchange and learning opportunities"]//parent::li//child::p')
# print(match_cultural_exchange.text)
# match_help = driver.find_element(By.XPATH, '//img[@alt="Help"]//parent::li//child::p')
# print(match_help.text)
# match_languages_spoken = driver.find_element(By.XPATH, '//h2[contains(text(), "Languages")]')
# print(match_languages_spoken.text)
# match_accommodation = driver.find_element(By.XPATH, '//img[@alt="Accommodation"]//parent::li//child::p')
# print(match_accommodation.text)
# try:
#     #match_what_else = driver.find_element(By.XPATH, '//img[@alt="What else ..."]//parent::li//child::p')
#     match_what_else = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, '//img[@alt="What else ..."]//parent::li//child::p'))).text
# except TimeoutException:
#        print("TimeoutException: what_else paragraph not found")
#        match_what_else = None
# print(match_what_else)
# match_internet_access = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[text()="Internet access"]//parent::div//parent::li//img[contains(@src, "")]')))
# print(match_internet_access.get_attribute("src"))
# match_limited_internet = driver.find_element(By.XPATH, '//*[text()="Limited internet access"]//parent::div//parent::li//child::img')
# print(match_limited_internet.get_attribute("src"))
# match_pets = driver.find_element(By.XPATH, '//*[text()="We have pets"]//parent::div//parent::li//child::img')
# print(match_pets.get_attribute("src"))
# match_smokers = driver.find_element(By.XPATH, '//*[text()="We are smokers"]//parent::div//parent::li//child::img')
# print(match_smokers.get_attribute("src"))
# match_hosting_families = driver.find_element(By.XPATH, '//*[text()="Can host families"]//parent::div//parent::li//child::img')
# print(match_hosting_families.get_attribute("src"))
# try:
#     match_digital_nomad = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, '//*[text()="Can host digital nomads"]//parent::div//child::p'))).text
#     #match_digital_nomad = driver.find_element(By.XPATH, '//*[text()="Can host digital nomads"]//parent::div//child::p')  #old version without wait
# except TimeoutException:
#        print("TimeoutException: digital_nomad paragraph not found")
#        match_digital_nomad = None
# digital_nomad.append(match_digital_nomad)
# print(digital_nomad)
# try:
#     match_campervans = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, '//*[text()="Space for parking camper vans"]//parent::div//child::p'))).text
#     #match_campervans = driver.find_element(By.XPATH, '//*[text()="Space for parking camper vans"]//parent::div//child::p').text
# except TimeoutException:
#     print("TimeoutException: campervans paragraph not found")
#     match_campervans = None
# campervans.append(match_campervans)
# print(match_campervans)
# possibly_pets = [] #
# try:
#     match_possibly_pets = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, '//*[text()="Can possibly accept pets"]//parent::div//child::p'))).text
#     #match_pets = driver.find_element(By.XPATH, '//*[text()="Can possibly accept pets"]//parent::div//child::p') #old version
# except TimeoutException:
#     print("TimeoutException: pets paragraph not found")
#     match_possibly_pets = None
# possibly_pets.append(match_possibly_pets)
# print(match_possibly_pets)
# match_workawayer_total = driver.find_element(By.XPATH, '//*[text()="How many Workawayers can stay?"]//parent::div//child::p')
# print(match_workawayer_total.text)
# match_hours_expected = driver.find_element(By.XPATH, '//*[text()="Hours expected"]//parent::div//child::p')
# print(match_hours_expected.text)
#match_number_of_photos = len(driver.find_elements(By.XPATH, '//div[contains(@id, "pic")]'))
# match_feedback_accuracy = driver.find_element(By.XPATH, '(//*[text()="These are extra optional ratings when members leave feedback. The average rating left for each option is displayed."]//following::p)[1]')
# print(match_feedback_accuracy.text)
# match_cultural_exchange = driver.find_element(By.XPATH, '(//*[text()="These are extra optional ratings when members leave feedback. The average rating left for each option is displayed."]//following::p)[2]')
# print(match_cultural_exchange.text)
# match_communication = driver.find_element(By.XPATH, '(//*[text()="These are extra optional ratings when members leave feedback. The average rating left for each option is displayed."]//following::p)[3]')
# print(match_communication.text)

       country      continent
107    Albania         Europe
108    Andorra         Europe
109    Austria         Europe
110    Belarus         Europe
111    Belgium         Europe
..         ...            ...
217   Paraguay  South America
218       Peru  South America
219   Suriname  South America
220    Uruguay  South America
221  Venezuela  South America

[115 rows x 2 columns]
printing just before sending the command to searchbox Albania
No Visa popup to handle
No Visa popup to handle
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comment

In [26]:
#Workaway_single_country.py
from pickle import FALSE
import time
from operator import contains
from select import select
from tomlkit import key_value
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import numpy as np

#DIFFERENT METHODS OF ACCESSING ELEMENTS
#print(match_internet_access.get_attribute('innerHTML'))
#print(match_internet_access.get_attribute('innerText'))
# print(match_internet_access.get_property)
# print(match_internet_access.text)
# print(match_internet_access.accessible_name)
# print(match_internet_access.tag_name)
### Scroll the page
#driver.execute_script("window.scrollTo(0,1800)")
### Handling objects in frames (iframe or frame doesn't matter)
#iframe = driver.find_element(By.XPATH, "//iframe[contains(@name, 'uspapi')]")
#driver.switch_to.frame(iframe)
### Find elements by Class name
#match_host_rating = driver.find_element(By.CLASS_NAME, 'hr_100 inline-block')


##### INPUT AREA #####
website = 'https://www.workaway.info/en/hostlist'
countries = pd.read_csv("Countries.csv") #comment out if you want single country
#country = "Australia" #TBD = {/Ireland, United Kingdom, Chile, New Zealand, Colombia, Portugal, Germany, Italy, Canada, /Australia, /United States, /France, /Spain}
feedback = 1
lastminute = 0
##### END OF INPUT AREA #####

def configure_driver():
    chrome_options = webdriver.ChromeOptions()
    ### Disable displaying images
    #prefs = {"profile.managed_default_content_settings.images": 2}
    #chrome_options.add_experimental_option("prefs", prefs)
    ### Maximize window
    chrome_options.add_argument("--start-maximized")
    ### Add the argument and make the browser Headless.
    chrome_options.add_argument("--headless")
    ### Instantiate the Webdriver
    driver = webdriver.Chrome(options = chrome_options)
    return driver

### Extracting function on hostlist page
def extract_element(driver, 
                    my_timer, 
                    element, 
                    path, 
                    extract_attribute = 'text', 
                    attribute_version = 1, 
                    attribute_value = "src"):
    if attribute_version == 0:
        try:
            match_temp = WebDriverWait(driver, my_timer).until(EC.presence_of_all_elements_located((By.XPATH, path)))
            match = len(match_temp)
        except TimeoutException:
            print("TimeoutException: photos not found")
            match = np.nan
    elif attribute_version == 3:
        match = []
        try:
            match_temp = WebDriverWait(driver, my_timer).until(EC.presence_of_all_elements_located((By.XPATH, path)))
            for match_i in match_temp:
                match.append(getattr(match_i, extract_attribute))
        except TimeoutException:
            print("TimeoutException: comments not found")
            match = np.nan

    else:
        try:
            match_temp = WebDriverWait(driver, my_timer).until(EC.presence_of_element_located((By.XPATH, path)))
            if attribute_version == 2:
                try:
                    match = getattr(match_temp, extract_attribute)(attribute_value)
                except:
                    print("New error '2' to handle")
            elif attribute_version == 1:
                try:
                    match = getattr(match_temp, extract_attribute)
                except:
                    print("New error '1' to handle")
        except TimeoutException:
            #print("TimeoutException: paragraph not found")
            #print(f"{element=}")
            match = np.nan
    element.append(match)
    #print(element)

### Open the Workaway finding host website
driver = configure_driver()
driver.get(website)

### COOKIES ACCEPT
cookie_accept = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'CybotCookiebotDialogBodyButtonAccept')))
#cookie_accept = driver.find_element(By.ID, 'CybotCookiebotDialogBodyButtonAccept') #old version
driver.execute_script("arguments[0].click();", cookie_accept)

### Check "only with feedback" checkbox
if feedback == 1 and (driver.find_element(By.ID, 'with_feedback').is_selected()) != True:
    driver.find_element(By.ID, 'with_feedback').is_selected()
    feedback_checkbox = driver.find_element('xpath', '//label[@for="with_feedback"]/descendant::div')
    feedback_checkbox.click()


### LOOP THROUUGH COUNTRIES IN THE CSV LIST
for country in countries['country']: #comment it out if you want single country
    try:
        searchbar = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//input[@placeholder="Anywhere"]')))
    except TimeoutException:
        print(f'INFO: Looks like you broken the site during {country} search')
        driver.quit() #this was added to close the previous driver - TO BE TESTED
        driver.get(website)
        #continue
    searchbar.click()
    time.sleep(2)
    print(f'printing just before sending the command to searchbox {country}')
    searchbar.clear()
    searchbar.send_keys(country) 
    time.sleep(2)
    searchbar.send_keys(Keys.ARROW_DOWN)
    searchbar.send_keys(Keys.ENTER)
    ### Handling of optional visa popup - United Kingdom and United States //newly added try/except block
    try:
        visabutton = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '//button[@data-action="visaagree"]')))
        visabutton.click()
    except TimeoutException:
        print("No Visa popup to handle")    ### Find first host on the Listentry Title object and open it

    ### Lists initialization
    title = [] #
    country_host = [] #
    host_rating = [] #
    feedback_no = [] #
    h_comm = [] #
    ww_comm = [] #
    last_activity = [] #
    reply_rate = [] #
    average_reply_time = [] #
    favourited = [] #
    min_stay = [] #
    description = [] #
    types_of_help = [] #
    cultural_exchange = [] #
    help = [] #
    languages_spoken = [] #
    accommodation = [] #
    what_else = [] #
    internet_access = [] #
    limited_internet = [] #
    pets = [] #
    smokers = [] #
    hosting_families = [] #
    digital_nomad = [] #
    campervans = [] #
    possibly_pets = [] #
    workawayer_total = [] #
    hours_expected = [] #
    number_of_photos = [] #
    feedback_accuracy = [] #
    feedback_cultural_exchange = [] #
    feedback_communication = [] #

    ### WEBSCRAPING THE HOST DATA ###
    ### For LOOP to iterate over consecutive pages and the on page list of Listentry object which contains always 20hosts / 1 page
    j = 1
    while j < 400:  
        ### Find first host on the Listentry Title object and open it
        i = 1
        while i < 21:
            path_combined = f'(//div[@class="listentry-content"]/child::a)[{i}]'
            #print(path_combined)
            #(//div[@class="listentry-content"])[20]
            #'//div[@class="listentry-content"]/child::a'
            try:
                hostlist = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, path_combined)))
            except TimeoutException:
                print("TimeoutException: END OF HOST LIST")
                break
            driver.execute_script("arguments[0].click();", hostlist)
            #hostlist.click()
            ### Handling opened tabs
            p = driver.window_handles[0]
            #obtain browser tab window
            c = driver.window_handles[1]
            driver.switch_to.window(c)
            
            extract_element(driver, 10, title, '//div[@class="details"]//child::h1', 'text')
            extract_element(driver, 1, country_host, '//div[@class="country-row"]//child::p', 'text')
            extract_element(driver, 1, host_rating, '//div[contains(@class, "hr")]', 'get_attribute', attribute_version = 2, attribute_value = "class")
            extract_element(driver, 1, feedback_no, '//h2[text()="Feedback"]//parent::div//following-sibling::div[1]//p', 'text')
            extract_element(driver, 1, last_activity, '//*[text()="Last activity"]//following::div[1]//child::p', 'text')
            extract_element(driver, 1, reply_rate, '//*[text()="Reply rate"]//following::div[1]//child::p', 'text')
            extract_element(driver, 1, average_reply_time, '//*[text()="Average reply time:"]//following::div[1]//child::p', 'text')
            extract_element(driver, 1, favourited, '//p[@class="blue-text"]', 'text')
            extract_element(driver, 1, min_stay, '//p[@class="text-muted"]', 'text')
            extract_element(driver, 1, description, '(//div[@id="section-information"]//descendant::p)[1]', 'text')
            extract_element(driver, 1, types_of_help, '(//div[@id="section-information"]//descendant::p)[2]', 'text')
            extract_element(driver, 1, internet_access, '//*[text()="Internet access"]//parent::div//parent::li//img[contains(@src, "")]', 'get_attribute', attribute_version = 2, attribute_value = "src")
            extract_element(driver, 1, cultural_exchange, '//img[@alt="Cultural exchange and learning opportunities"]//parent::li//child::p', 'text')
            extract_element(driver, 1, help, '//img[@alt="Help"]//parent::li//child::p', 'text')
            extract_element(driver, 1, languages_spoken, '//h2[contains(text(), "Languages")]//parent::div//child::p', 'text')
            extract_element(driver, 1, accommodation, '//img[@alt="Accommodation"]//parent::li//child::p', 'text')
            extract_element(driver, 1, what_else, '//img[@alt="What else ..."]//parent::li//child::p', 'text')
            extract_element(driver, 1, limited_internet, '//*[text()="Limited internet access"]//parent::div//parent::li//child::img', 'get_attribute', attribute_version = 2, attribute_value = "src")
            extract_element(driver, 1, pets, '//*[text()="We have pets"]//parent::div//parent::li//child::img', 'get_attribute', attribute_version = 2, attribute_value = "src")
            extract_element(driver, 1, smokers, '//*[text()="We are smokers"]//parent::div//parent::li//child::img', 'get_attribute', attribute_version = 2, attribute_value = "src")
            extract_element(driver, 1, hosting_families, '//*[text()="Can host families"]//parent::div//parent::li//child::img', 'get_attribute', attribute_version = 2, attribute_value = "src")
            extract_element(driver, 1, digital_nomad, '//*[text()="Can host digital nomads"]//parent::div//child::p', 'text')
            extract_element(driver, 1, campervans, '//*[text()="Space for parking camper vans"]//parent::div//child::p', 'text')
            extract_element(driver, 1, possibly_pets, '//*[text()="Can possibly accept pets"]//parent::div//child::p', 'text')
            extract_element(driver, 1, workawayer_total, '//*[text()="How many Workawayers can stay?"]//parent::div//child::p', 'text')
            extract_element(driver, 1, hours_expected, '//*[text()="Hours expected"]//parent::div//child::p', 'text')
            extract_element(driver, 1, number_of_photos, '//div[contains(@id, "pic")]', attribute_version = 0)
            extract_element(driver, 1, feedback_accuracy, '(//*[text()="These are extra optional ratings when members leave feedback. The average rating left for each option is displayed."]//following::p)[1]', 'text')
            extract_element(driver, 1, feedback_cultural_exchange, '(//*[text()="These are extra optional ratings when members leave feedback. The average rating left for each option is displayed."]//following::p)[2]', 'text')
            extract_element(driver, 1, feedback_communication, '(//*[text()="These are extra optional ratings when members leave feedback. The average rating left for each option is displayed."]//following::p)[3]', 'text')
            extract_element(driver, 1, ww_comm, '//div[@class="feedback_content feedback_content_ww"]/parent::div/following-sibling::div[@class="feedback-thumb-wrapper"]/span', 'text', attribute_version = 3)
            extract_element(driver, 1, h_comm, '//div[@class="feedback_content feedback_content_h"]/parent::div/following-sibling::div[@class="feedback-thumb-wrapper"]/span', 'text', attribute_version = 3)

            
            driver.close()
            driver.switch_to.window(p)
            i += 1
        try:
            next_page = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//li[@class="pagination-custom-next"]//a')))
        except:
            print("Next button not found")
            break
        next_page.click()
        j += 1

    print(f"Total number of hosts webscraped in {country} = {i*j}")
    ### Creating DF and exporting to ';' delimited CSV file
    df = pd.DataFrame({'title': title,
        'country_host': country_host,
        'host_rating': host_rating,
        'ww_comm': ww_comm,
        'h_comm': h_comm,
        'feedback_no': feedback_no,
        'last_activity': last_activity,
        'reply_rate': reply_rate,
        'average_reply_time': average_reply_time,
        'favourited': favourited,
        'min_stay': min_stay,
        'description': description,
        'types_of_help': types_of_help,
        'cultural_exchange': cultural_exchange,
        'help': help,
        'languages_spoken': languages_spoken,
        'accomodation': accommodation,
        'what_else': what_else,
        'internet_access': internet_access,
        'limited_internet': limited_internet,
        'pets': pets, 
        'smokers': smokers,
        'hosting_families': hosting_families,
        'digital_nomad': digital_nomad,  
        'campervans':  campervans,
        'possibly_pets': possibly_pets,
        'workawayer_total': workawayer_total,
        'hours_expected': hours_expected,
        'number_of_photos': number_of_photos,
        'feedback_accuracy': feedback_accuracy,
        'feedback_cultural_exchange': feedback_cultural_exchange,
        'feedback_communication': feedback_communication})
    #print(df)
    saving_path = f'workaway_scraping_{country}.csv'
    df.to_csv(saving_path, sep =';', index = False)
driver.quit()

#2400 records in France in 365m

printing just before sending the command to searchbox Chile
No Visa popup to handle
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments not found
TimeoutException: comments n

KeyboardInterrupt: 